The purpose of this analysis is to perform some exploratory data analysis of NBA offense in the 2022-2023 season.

The nba API breaks down offensive possessions into several types :

* Cut
* Handoff
* Isolation
* Miscellaneous
* OffScreen
* Postup
* PRBallHandler
* PRRollman
* OffRebound
* Spotup
* Transition

A good starting point for measuring offensive efficiency is points per possession (PPP). To put our analysis into context, let's start by calculating the average points per possession across all players and play types.

The NBA SynergyPlayTypes endpoint doesn't let us call all players and play types at once, so instead we'll just do an API call for each playtype and merge the datasets together.

In [28]:
from nba_api.stats import endpoints
import pandas as pd
import time

# loop over the different play types
play_types_list = ['Cut','Handoff','Isolation','Misc','OffScreen','Postup','PRBallHandler','PRRollman','OffRebound','Spotup','Transition']
df = pd.DataFrame()

for play_type in play_types_list:

    # The NBA API often times out on calls, using a while loop to simply automate retries of our api call
    while True:
        try:
            response = endpoints.SynergyPlayTypes(play_type_nullable=play_type, player_or_team_abbreviation='P', type_grouping_nullable='Offensive').get_data_frames()[0]
            print(f'{play_type} called successfully.')
            break # quit the loop if successful
        except:
            print(f'Error with {play_type} call.')
            time.sleep(5)

    # combine each call into one dataframe
    df = pd.concat([df,response])
    time.sleep(5)

df.to_csv('synergy_all_offensive_possessions_by_player.csv')

Cut called successfully.
Handoff called successfully.
Isolation called successfully.
Error with Misc call.
Misc called successfully.
OffScreen called successfully.
Error with Postup call.
Postup called successfully.
PRBallHandler called successfully.
PRRollman called successfully.
OffRebound called successfully.
Spotup called successfully.
Transition called successfully.


In [30]:
df = df[['PLAYER_NAME','TEAM_NAME','PLAY_TYPE','PERCENTILE','PPP','POSS']]

In [31]:
df

,PLAYER_NAME,TEAM_NAME,PLAY_TYPE,PERCENTILE,PPP,POSS
0,Jarrett Allen,Cleveland Cavaliers,Cut,0.592,1.338,207
1,Domantas Sabonis,Sacramento Kings,Cut,0.551,1.328,195
2,Nic Claxton,Brooklyn Nets,Cut,0.650,1.365,181
3,Evan Mobley,Cleveland Cavaliers,Cut,0.803,1.453,161
4,Rudy Gobert,Minnesota Timberwolves,Cut,0.680,1.377,154
...,...,...,...,...,...,...
380,Kendrick Nunn,Washington Wizards,Transition,0.030,0.615,13
381,Caleb Houstan,Orlando Magic,Transition,0.015,0.462,13
382,Isaiah Hartenstein,New York Knicks,Transition,0.008,0.400,15
383,Chuma Okeke,Orlando Magic,Transition,0.008,0.400,15
